In [1]:
import numpy as np;
import pandas as pd;
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef,make_scorer
mccScorer = make_scorer(matthews_corrcoef)
from numpy import mean
from numpy import std

from google.colab import drive
drive.mount("/content/gdrive")

# ANOVA feature selection for numeric input and categorical output
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import discriminant_analysis
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

Mounted at /content/gdrive


In [2]:
#read training dataset 1
data1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_1_Training.csv')

X_train_actual_1=data1[:-2].T[1:].values

y_train_actual_1 = data1[-2:].T[-130:].T.values

In [3]:
#read training dataset 2
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_2_Training.csv')

X_train_actual_2=data2[:-4].T[1:].values

y_train_actual_2 = data2[-4:].T[-340:].T.values

In [4]:
#read testing dataset 1
data_test_1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_1_Testing.csv')

X_test_1 = data_test_1.T[1:].values

print(X_test_1.shape)

(100, 22283)


In [5]:
#read testing dataset 2
data_test_2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_2_Testing.csv')

X_test_2 = data_test_2.T[1:].values

print(X_test_2.shape)

(214, 54675)


In [6]:
co1={'X':X_train_actual_1,'Y': y_train_actual_1[0]};
co2={'X':X_train_actual_1,'Y': y_train_actual_1[1]};

In [7]:
co3={'X':X_train_actual_2,'Y': y_train_actual_2[0]};
co4={'X':X_train_actual_2,'Y': y_train_actual_2[1]};
co5={'X':X_train_actual_2,'Y': y_train_actual_2[2]};
co6={'X':X_train_actual_2,'Y': y_train_actual_2[3]};

In [8]:
def determine_number_of_top_features(X,Y):
  X=pd.DataFrame(X)
  X=X.astype(float)

  Y=pd.DataFrame(Y)
  Y=Y.astype(int)

  sel = f_classif(X, Y)
  p_values = pd.Series(sel[1])
  p_values.index = X.columns
  p_values.sort_values(ascending=True, inplace=True)
  p_values = p_values[p_values < 0.05]
  X_p = X[p_values.index]
  return X_p.shape[1]

In [9]:
best_co1=determine_number_of_top_features(**co1)
best_co2=determine_number_of_top_features(**co2)
best_co3=determine_number_of_top_features(**co3)
best_co4=determine_number_of_top_features(**co4)
best_co5=determine_number_of_top_features(**co5)
best_co6=determine_number_of_top_features(**co6)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [10]:
print(best_co1)
print(best_co2)
print(best_co3)
print(best_co4)
print(best_co5)
print(best_co6)

3923
2734
9607
6773
3522
4008


In [11]:
def feature_selector(X,Y,numberOfFeaturesToBeSelected=1):
  if(numberOfFeaturesToBeSelected == 1):
    totalNumberOfFeatures=X.shape[1]
    numberOfFeaturesToBeSelected=totalNumberOfFeatures//2;
  # define feature selection
  fs = SelectKBest(score_func=f_classif, k=numberOfFeaturesToBeSelected)
  # apply feature selection
  return fs.fit(X, Y)

In [12]:
def SVM_SVC_pure_model(X,Y,C,gamma,kernel,degree,prefit_feature_model):
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X.astype(np.float32))
  
  # define model
  model = SVC(cache_size=2000,class_weight='balanced',C=C,gamma=gamma,kernel=kernel,degree=degree)
  
  # select features from it
  X_selected_features = prefit_feature_model.transform(X_train_scaled);
  
  #Fit the classifier on selected features
  model.fit(X_selected_features,Y.astype('int'))
  return model;

In [13]:
def feature_selection_SVM_SVC_predict(X,Y,C,gamma,kernel,degree,X_test,number_of_features=1):
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y,numberOfFeaturesToBeSelected=number_of_features);
    cModel = SVM_SVC_pure_model(X,Y,C,gamma,kernel,degree,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred


In [14]:
def linearSVM_pure_model(X,Y,C,loss,penalty,prefit_feature_model):
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X.astype(np.float32))
  
  # define model
  model = LinearSVC(class_weight='balanced',C=C,loss=loss,penalty=penalty)
  
  # select features from it
  X_selected_features = prefit_feature_model.transform(X_train_scaled);
  
  #Fit the classifier on selected features
  model.fit(X_selected_features,Y.astype('int'))
  return model;

In [15]:
def feature_selection_LinearSVM_predict(X,Y,C,loss,penalty,X_test,number_of_features=1):
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y,numberOfFeaturesToBeSelected=number_of_features);
    cModel = linearSVM_pure_model(X,Y,C,loss,penalty,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred


In [16]:
def logisticRegClassifier_pure_model(X,Y,C,penalty,solver,prefit_feature_model):
    # define model
    model = LogisticRegression(C=C,penalty=penalty,solver=solver,class_weight='balanced')
    
    # select features from it
    X_selected_features = prefit_feature_model.transform(X);
    #Fit the classifier on selected features
    model.fit(X_selected_features,Y.astype('int'))
    return model;

In [17]:
def feature_selection_logRegClassification_predict(X,Y,C,penalty,solver,X_test,number_of_features=1):
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y,numberOfFeaturesToBeSelected=number_of_features);
    cModel = logisticRegClassifier_pure_model(X,Y,C,penalty,solver,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred


In [18]:
def stackingClassifier_pure_model(X,Y,level0,prefit_feature_model):
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X.astype(np.float32))
  
  # # define the base models
  # level0 = list()
  # level0.append(('lr', LogisticRegression(class_weight='balanced')))
  # level0.append(('knn', KNeighborsClassifier()))
  # # level0.append(('gd', GradientBoostingClassifier()))
  # # level0.append(('ada', AdaBoostClassifier()))
  # level0.append(('rf', RandomForestClassifier(class_weight='balanced',n_estimators=500,random_state=1)))
  # level0.append(('bayes', GaussianNB()))
  # define meta learner model
  level1 = LogisticRegression()
  # define the stacking ensemble
  model = StackingClassifier(estimators=level0, final_estimator=level1)
  
  # select features from it
  X_selected_features = prefit_feature_model.transform(X_train_scaled);
  #Fit the classifier on selected features
  model.fit(X_selected_features,Y.astype('int'))
  return model;


In [19]:
def feature_selection_stackingClassifier_predict(X,Y,level0,X_test,number_of_features=1):
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test.astype(np.float32))
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y,numberOfFeaturesToBeSelected=number_of_features);
    cModel = stackingClassifier_pure_model(X,Y,level0,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test_scaled)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred


In [20]:
def ADABoost_with_logregbase_pure_model(X,Y,n_estimator,learning_rate,prefit_feature_model,C=1,penalty='l2',solver='lbfgs'):
    base_estimator = LogisticRegression(C=C,penalty=penalty,solver=solver,class_weight='balanced')
    # define model
    model = AdaBoostClassifier(base_estimator=base_estimator,learning_rate=learning_rate,n_estimators=n_estimator)
    
    # select features from it
    X_selected_features = prefit_feature_model.transform(X);
    #Fit the classifier on selected features
    model.fit(X_selected_features,Y.astype('int'))
    return model;

In [21]:
def feature_selection_AdaLogReg_predict(X,Y,n_estimator,lerning_rate,X_test,C=1,penalty='l2',solver='lbfgs',number_of_features=1):
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y,numberOfFeaturesToBeSelected=number_of_features);
    cModel = ADABoost_with_logregbase_pure_model(X,Y,n_estimator,lerning_rate,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred


In [22]:
def LDA_pure_model(X,Y,shrinkage,solver,prefit_feature_model):
    # define model
    model = discriminant_analysis.LinearDiscriminantAnalysis(shrinkage= shrinkage, solver= solver)
    
    # select features from it
    X_selected_features = prefit_feature_model.transform(X);
    #Fit the classifier on selected features
    model.fit(X_selected_features,Y.astype('int'))
    return model;

In [23]:
def feature_selection_LDA_predict(X,Y,shrinkage,solver,X_test,number_of_features=1):
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y,numberOfFeaturesToBeSelected=number_of_features);
    cModel = LDA_pure_model(X,Y,shrinkage,solver,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred


In [24]:
def QDA_pure_model(X,Y,prefit_feature_model):
    # define model
    model = discriminant_analysis.QuadraticDiscriminantAnalysis()
    
    # select features from it
    X_selected_features = prefit_feature_model.transform(X);
    #Fit the classifier on selected features
    model.fit(X_selected_features,Y.astype('int'))
    return model;

In [25]:
def feature_selection_QDA_predict(X,Y,X_test,number_of_features=1):
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y,numberOfFeaturesToBeSelected=number_of_features);
    cModel = QDA_pure_model(X,Y,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred


In [26]:
def randomForestClassifier_pure_model(X,Y,n_estimators,prefit_feature_model):
    # define model
    model = RandomForestClassifier(n_estimators=n_estimators,class_weight='balanced')
    
    # select features from it
    X_selected_features = prefit_feature_model.transform(X);
    #Fit the classifier on selected features
    model.fit(X_selected_features,Y.astype('int'))
    return model;

In [27]:
def feature_selection_randomForestClassifier_predict(X,Y,n_estimator,X_test,number_of_features=1):
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y,numberOfFeaturesToBeSelected=number_of_features);
    cModel = randomForestClassifier_pure_model(X,Y,n_estimator,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred

In [28]:
def simple_ADABoost_fit_predict(X,Y,lr,X_test):
    base_estimator = DecisionTreeClassifier(max_depth=3,class_weight='balanced')
    model = AdaBoostClassifier(base_estimator=base_estimator,learning_rate=lr,n_estimators=50)
    
    #Fit the classifier on selected features
    model.fit(X,Y.astype('int'))

    y_test_pred=model.predict(X_test)

    return y_test_pred

In [29]:
# THis SVM was not classifying all same class
#feature_selection_SVM_SVC_predict(co1['X'],co1['Y'],100,'scale','poly',3,X_test_1)


In [30]:
#feature_selection_SVM_SVC_predict(co6['X'],co6['Y'],50,'scale','rbf',1,X_test_2,best_co6)

In [39]:
def predict_all_endpoints():
  master_y_pred_out = np.array([])
  # Here SVM-SVC and linear SVM predicts all 1s or all 0s.
  # Here LDA and Log-reg showed exact same performance in kaggle
  print("CO1-pred started")
  level0 = list()
  # level0.append(('lr',LogisticRegression(class_weight='balanced') ))
  # level0.append(('lda',discriminant_analysis.LinearDiscriminantAnalysis(shrinkage= 'auto', solver= 'lsqr') ))
  # level0.append(('knn', KNeighborsClassifier()))
  # level0.append(('gd', GradientBoostingClassifier()))
  # level0.append(('ada', AdaBoostClassifier(base_estimator=base_estimator,learning_rate=0.1,n_estimators=50)))
  # level0.append(('rf', RandomForestClassifier(class_weight='balanced',n_estimators=500,random_state=1)))
  # level0.append(('bayes', GaussianNB()))
  #pred=feature_selection_LinearSVM_predict(co1['X'],co1['Y'],10,'squared_hinge','l2',X_test_1);
  # pred=feature_selection_stackingClassifier_predict(co1['X'],co1['Y'],level0,X_test_1,best_co1)
  #pred = feature_selection_AdaLogReg_predict(co1['X'],co1['Y'],50,0.1,X_test_1,1e-05, 'none','newton-cg')
  #pred = feature_selection_SVM_SVC_predict(co1['X'],co1['Y'],100,'scale','rbf',X_test_1)
  #pred = feature_selection_logRegClassification_predict(co1['X'],co1['Y'],1e-05, 'none','newton-cg',X_test_1)
  ##### LR with above config and LDA with below config perform equally on kaggle public data. LDA has a better ROC score and little more mean MCC. So better to use that instead of LR
  pred = feature_selection_LDA_predict(co1['X'],co1['Y'],'auto','lsqr',X_test_1,best_co1)

  master_y_pred_out=np.append(master_y_pred_out,pred);

  # Here SVM-SVC predicts all 1s or all 0s.
  print("CO2-pred started")
  # pred = feature_selection_LDA_predict(co2['X'],co2['Y'],'auto','lsqr',X_test_1,best_co2)
  pred = feature_selection_logRegClassification_predict(co2['X'],co2['Y'],0.1, 'l1','liblinear',X_test_1)
  #pred=feature_selection_stackingClassifier_predict(co2['X'],co2['Y'],X_test_1,best_co2)
  #pred = feature_selection_AdaLogReg_predict(co2['X'],co2['Y'],50,0.1,X_test_1)
  # pred= feature_selection_randomForestClassifier_predict(co2['X'],co2['Y'],800,X_test_1,20)
  #pred = feature_selection_SVM_SVC_predict(co2['X'],co2['Y'],0.1,'scale','rbf',X_test_1)
  master_y_pred_out=np.append(master_y_pred_out,pred);
  
  print("CO3-pred started")
  pred = simple_ADABoost_fit_predict(co3['X'],co3['Y'],0.1,X_test_2)
  #pred=feature_selection_stackingClassifier_predict(co3['X'],co3['Y'],X_test_2,best_co3)
  # pred = feature_selection_logRegClassification_predict(co3['X'],co3['Y'],1e-05, 'none','newton-cg',X_test_2)
  #pred = feature_selection_SVM_SVC_predict(co3['X'],co3['Y'],1,'scale','rbf',X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  print("CO4-pred started")
  pred = simple_ADABoost_fit_predict(co4['X'],co4['Y'],0.1,X_test_2)
  # pred = feature_selection_logRegClassification_predict(co4['X'],co4['Y'],1, 'l2','lbfgs',X_test_2)
  #pred=feature_selection_stackingClassifier_predict(co4['X'],co4['Y'],X_test_2,best_co4)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  # This works well for CO5- Don't touch
  print("CO5-pred started")
  pred = simple_ADABoost_fit_predict(co5['X'],co5['Y'],0.1,X_test_2)
  # pred = feature_selection_AdaLogReg_predict(co5['X'],co5['Y'],100,0.1,X_test_2,0.01, 'l1','liblinear',best_co5)
  #pred=feature_selection_stackingClassifier_predict(co5['X'],co5['Y'],X_test_2,best_co5)
  # pred = feature_selection_logRegClassification_predict(co5['X'],co5['Y'],0.01, 'l1','liblinear',X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);
  ######################################

  # Here SVM-SVC predicts all 1s or all 0s.
  print("CO6-pred started")
  # define the base models
  level0 = list()
  # level0.append(('lr',LogisticRegression(class_weight='balanced') ))
  # level0.append(('knn', KNeighborsClassifier()))
  # level0.append(('gd', GradientBoostingClassifier()))
  # level0.append(('ada', AdaBoostClassifier(learning_rate=0.1,n_estimators=50)))
  # level0.append(('rf', RandomForestClassifier(class_weight='balanced',n_estimators=500,random_state=1)))
  # level0.append(('bayes', GaussianNB()))
  # pred=feature_selection_stackingClassifier_predict(co6['X'],co6['Y'],level0,X_test_2,best_co6)
  pred = simple_ADABoost_fit_predict(co6['X'],co6['Y'],0.1,X_test_2)
  # pred = feature_selection_QDA_predict(co6['X'],co6['Y'],X_test_2)
  # pred = feature_selection_logRegClassification_predict(co6['X'],co6['Y'],1e-05, 'l2','lbfgs',X_test_2)
  #pred = feature_selection_AdaLogReg_predict(co6['X'],co6['Y'],100,0.1,X_test_2)
  #pred = feature_selection_LDA_predict(co6['X'],co6['Y'],'auto','lsqr',X_test_2,(co6['X'].shape[1] // 10))
  #pred = feature_selection_randomForestClassifier_predict(co6['X'],co6['Y'],800,X_test_2,50)
  #pred = feature_selection_SVM_SVC_predict(co6['X'],co6['Y'],0.1,'scale','rbf',X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  return master_y_pred_out

In [40]:
def predict_all_endpoints_output_to_csv():
  total_pred = predict_all_endpoints()

  np.savetxt("reproduce-0.5.csv", np.dstack((np.arange(0, total_pred.size),total_pred))[0],"%d,%d",header="Id,Predicted")

In [41]:
predict_all_endpoints_output_to_csv()

CO1-pred started
CO2-pred started
CO3-pred started
CO4-pred started
CO5-pred started
CO6-pred started


In [ ]:
def best_simple_ADABoost_fit_predict(X,Y,lr,X_test):
    base_estimator = DecisionTreeClassifier(max_depth=3,class_weight='balanced')
    model = AdaBoostClassifier(base_estimator=base_estimator,learning_rate=lr)
    
    #Fit the classifier on selected features
    model.fit(X,Y.astype('int'))

    y_test_pred=model.predict(X_test)

    return y_test_pred

In [ ]:
#0.5 MCC
def best5MCC_predict_all_endpoints():
  master_y_pred_out = np.array([])
  # Here SVM-SVC and linear SVM predicts all 1s or all 0s.
  # Here LDA and Log-reg showed exact same performance in kaggle
  print("CO1-pred started")
  pred = feature_selection_LDA_predict(co1['X'],co1['Y'],'auto','lsqr',X_test_1,best_co1)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  # Here SVM-SVC predicts all 1s or all 0s.
  print("CO2-pred started")
  pred = feature_selection_logRegClassification_predict(co2['X'],co2['Y'],0.1, 'l1','liblinear',X_test_1)
  master_y_pred_out=np.append(master_y_pred_out,pred);
  
  print("CO3-pred started")
  pred = best_simple_ADABoost_fit_predict(co3['X'],co3['Y'],0.1,X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  print("CO4-pred started")
  pred = best_simple_ADABoost_fit_predict(co4['X'],co4['Y'],0.1,X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  # This works well for CO5- Don't touch
  print("CO5-pred started")
  pred = best_simple_ADABoost_fit_predict(co5['X'],co5['Y'],0.1,X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);
  ######################################

  # Here SVM-SVC predicts all 1s or all 0s.
  print("CO6-pred started")
  pred = best_simple_ADABoost_fit_predict(co6['X'],co6['Y'],0.1,X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  return master_y_pred_out

In [ ]:
#best so far (MCC-0.4451)
def best_predict_all_endpoints():
  master_y_pred_out = np.array([])
  print("CO1-pred started")
  pred = feature_selection_logRegClassification_predict(co1['X'],co1['Y'],1e-05, 'none','newton-cg',X_test_1)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  print("CO2-pred started")
  pred = feature_selection_logRegClassification_predict(co2['X'],co2['Y'],0.1, 'l1','liblinear',X_test_1)
  master_y_pred_out=np.append(master_y_pred_out,pred);
  
  print("CO3-pred started")
  pred = feature_selection_logRegClassification_predict(co3['X'],co3['Y'],1e-05, 'none','newton-cg',X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  print("CO4-pred started")
  pred = feature_selection_logRegClassification_predict(co4['X'],co4['Y'],1, 'l2','lbfgs',X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  # This works well for CO5- Don't touch
  print("CO5-pred started")
  pred = feature_selection_logRegClassification_predict(co5['X'],co5['Y'],0.01, 'l1','liblinear',X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);
  ######################################

  print("CO6-pred started")
  pred = feature_selection_logRegClassification_predict(co6['X'],co6['Y'],1e-05, 'l2','lbfgs',X_test_2)
  master_y_pred_out=np.append(master_y_pred_out,pred);

  return master_y_pred_out

In [ ]:
def best_predict_all_endpoints_output_to_csv():
  total_pred = best_predict_all_endpoints()

  np.savetxt("best-prediction_CO-5.csv", np.dstack((np.arange(0, total_pred.size),total_pred))[0],"%d,%d",header="Id,Predicted")

In [ ]:
#best_predict_all_endpoints_output_to_csv()

In [ ]:
def logisticRegClassifier_forCO5(X,Y,C,penalty,solver):
  #logisticRegClassifier(co5['X'],co5['Y'],0.01, 'l1','liblinear')
  
  scoringReports={'mccScorer': mccScorer,'accuracy': 'accuracy','precision':'precision','recall':'recall', 'f1':'f1'}
  
  # define model
  model = LogisticRegression(C=C,penalty=penalty,solver=solver,class_weight='balanced')
  # define evaluation
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)

  scores = cross_validate(model, X, Y.astype('int'), scoring=scoringReports, cv=cv)

  # summarize result
  print('MCC Score- Mean: %.3f SD: (%.3f)' % (mean(scores['test_mccScorer']), std(scores['test_mccScorer'])))
  print(list(scores.keys()))
  print("Means",mean(list(scores.values()),axis=1))
  print("SD",std(list(scores.values()),axis=1))

In [ ]:
def feature_selection_logRegClassification_predict_for_CO5(X,Y,C,penalty,solver,X_test):
    # get fitted feature selector model
    prefit_feature_model = feature_selector(X,Y);
    cModel = logisticRegClassifier_pure_model(X,Y,C,penalty,solver,prefit_feature_model)
    X_test_selected_features = prefit_feature_model.transform(X_test)
    y_test_pred=cModel.predict(X_test_selected_features)
    return y_test_pred

In [ ]:
def compute_class_frequency(Y):
  totalLen = len(Y)
  trueClassCount=np.sum(Y);
  flaseClassCount = totalLen-trueClassCount;
  print("True",trueClassCount)
  print("False", flaseClassCount)


def compute_class_frequency_of_all_classes():
  print("CO1-Class frequency")
  compute_class_frequency(co1['Y'])
  print("CO2-Class frequency")
  compute_class_frequency(co2['Y'])
  print("CO3-Class frequency")
  compute_class_frequency(co3['Y'])
  print("CO4-Class frequency")
  compute_class_frequency(co4['Y'])
  print("CO5-Class frequency")
  compute_class_frequency(co5['Y'])
  print("CO6-Class frequency")
  compute_class_frequency(co6['Y'])